# Spark Exercises

Within your codeup-data-science directory, create a new repo named spark-exercises. This will be where you do your work for this module. Create a repository on GitHub with the same name, and link your local repository to GitHub.

Save this work in your spark-exercises repo. Then add, commit, and push your changes.

Create a jupyter notebook or python script named spark101 for this exercise.

### Imports

In [1]:
import pandas as pd
import numpy as np
#create the spark session
import pyspark

spark = pyspark.sql.SparkSession.builder.getOrCreate()